In [9]:
!pip install simplekml


[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import geopy.distance
from simplekml import Kml

# Original coordinate (latitude, longitude)
origin = (17.117113, 79.837709)

# Convert 10 feet to meters (1 foot = 0.3048 meters)
step = 10 * 0.3048

# 1 hectare = 10000 square meters
# Area of 1 hectare in meters is a square of side length 100 meters
side_length = 100

# Number of steps in one direction (both latitude and longitude)
steps = int(side_length // step)

# List to store generated coordinates
coordinates = []

for i in range(steps + 1):
    for j in range(steps + 1):
        # Calculate new latitude and longitude
        lat_shift = geopy.distance.distance(meters=i*step).destination(origin, 0).latitude - origin[0]
        lon_shift = geopy.distance.distance(meters=j*step).destination(origin, 90).longitude - origin[1]
        new_lat = origin[0] + lat_shift
        new_lon = origin[1] + lon_shift
        coordinates.append((new_lat, new_lon))

for coord in coordinates:
    print (coord)

# Create a KML file
kml = Kml()
for coord in coordinates:
    kml.newpoint(coords=[(coord[1], coord[0])])

# Save the KML file
#kml.save("coordinates.kml")

print("KML file has been created.")


(17.117113000000003, 79.837709)
(17.117113000000003, 79.83773764137489)
(17.117113000000003, 79.83776628274975)
(17.117113000000003, 79.83779492412464)
(17.117113000000003, 79.8378235654995)
(17.117113000000003, 79.83785220687439)
(17.117113000000003, 79.83788084824926)
(17.117113000000003, 79.83790948962414)
(17.117113000000003, 79.837938130999)
(17.117113000000003, 79.83796677237389)
(17.117113000000003, 79.83799541374876)
(17.117113000000003, 79.83802405512364)
(17.117113000000003, 79.83805269649851)
(17.117113000000003, 79.83808133787339)
(17.117113000000003, 79.83810997924826)
(17.117113000000003, 79.83813862062314)
(17.117113000000003, 79.83816726199801)
(17.117113000000003, 79.83819590337289)
(17.117113000000003, 79.83822454474776)
(17.117113000000003, 79.83825318612264)
(17.117113000000003, 79.83828182749751)
(17.117113000000003, 79.8383104688724)
(17.117113000000003, 79.83833911024726)
(17.117113000000003, 79.83836775162214)
(17.117113000000003, 79.83839639299701)
(17.11711300

In [3]:
!pip install geopandas

  Obtaining dependency information for geopandas from https://files.pythonhosted.org/packages/c4/64/7d344cfcef5efddf9cf32f59af7f855828e9d74b5f862eddf5bfd9f25323/geopandas-1.0.1-py3-none-any.whl.metadata
  Obtaining dependency information for pyogrio>=0.7.2 from https://files.pythonhosted.org/packages/c3/fa/45efa8c96744ddd92c3ce3a80ddba8512954cc7c5407876e2ff2ffea0c10/pyogrio-0.9.0-cp312-cp312-win_amd64.whl.metadata
  Obtaining dependency information for pyproj>=3.3.0 from https://files.pythonhosted.org/packages/c7/f3/2f32fe143cd7ba1d4d68f1b6dce9ca402d909cbd5a5830e3a8fa3d1acbbf/pyproj-3.6.1-cp312-cp312-win_amd64.whl.metadata
  Obtaining dependency information for shapely>=2.0.0 from https://files.pythonhosted.org/packages/d4/c3/e98e3eb9f06def32b8e2454ab718cafb99149f023dff023e257125132d6e/shapely-2.0.5-cp312-cp312-win_amd64.whl.metadata
   ---------------------------------------- 0.0/323.6 kB ? eta -:--:--
   ---------- ----------------------------- 81.9/323.6 kB 2.3 MB/s eta 0:00:01
   -


[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import geopandas as gpd
from shapely.geometry import Polygon, Point
import geopy.distance
import numpy as np
from fastkml import kml

# Function to load and parse KML file
def load_kml(file_path):
    with open(file_path, 'rt', encoding='utf-8') as f:
        doc = f.read()
    k = kml.KML()
    k.from_string(doc)
    features = list(k.features())
    placemarks = list(features[0].features())
    polygons = [placemark.geometry for placemark in placemarks if isinstance(placemark.geometry, Polygon)]
    return polygons[0] if polygons else None

# Function to generate points within a polygon
def generate_points_within_polygon(polygon, step_size_feet=10):
    step_size_meters = step_size_feet * 0.3048  # Convert feet to meters

    min_x, min_y, max_x, max_y = polygon.bounds
    points = []

    # Generate a grid of points within the bounding box of the polygon
    x_coords = np.arange(min_x, max_x, step_size_meters)
    y_coords = np.arange(min_y, max_y, step_size_meters)

    for x in x_coords:
        for y in y_coords:
            point = Point(x, y)
            if polygon.contains(point):
                points.append(point)

    return points

# Function to convert generated points back to geographic coordinates
def convert_to_geographic(points, origin):
    geo_points = []
    for point in points:
        d = geopy.distance.distance(meters=point.x)
        new_lat = d.destination(origin, 0).latitude
        d = geopy.distance.distance(meters=point.y)
        new_lon = d.destination(origin, 90).longitude
        geo_points.append((new_lat, new_lon))
    return geo_points

# Main function to run the code
def main(kml_file_path, origin):
    polygon = load_kml(kml_file_path)
    if polygon is None:
        print("No polygon found in the KML file.")
        return

    # Generate points within the polygon
    points = generate_points_within_polygon(polygon)

    # Convert points to geographic coordinates based on the origin
    geo_points = convert_to_geographic(points, origin)

    # Print the generated coordinates
    for coord in geo_points:
        print(coord)

# Usage example
kml_file_path = r"C:\WORK\Data\Nursery\Codes\coordinates.kml"
origin = (17.117113, 79.837709)  # Use the same origin as before
main(kml_file_path, origin)


ModuleNotFoundError: No module named 'fastkml'